In [113]:
from typing import Dict, Tuple

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold

# visualization tools:
import matplotlib.pyplot as plt
import seaborn as sns

# models:
from xgboost import XGBClassifier

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

# evaluation functions:
from sklearn.metrics import precision_score, recall_score, f1_score



pd.set_option('display.max_columns', 100)


In [53]:
# notebook goal: Setup a basic machine learning framework that cleans data, standardizes features,
#  evaluates feature impt, shap values, and a myriad of ML algorithms
# TODO: add the day-of-week as a feature
# TODO: Add in target date versus historic reference dates
# TODO: Add in volume-based feature functionality
# TODO: Evaluate standardizing features per stock or one model per stock - may not be enough data realistically
# TODO: Check bol-range-pct calculation - only giving zero value
# TODO: Add profit point forecast

In [68]:
# functions:

def clean_stock_data(dataframe: pd.DataFrame) -> pd.DataFrame :

    '''removes nulls and in the future will be built out to do any additonal cleaning on the dataframe that is necessary
    Args:
        dataframe: pandas dataframe containing all of the potential features
        parameters: 
            calculation_field: field on which all of the features are built

    Returns:
        dataframe: dataset that is ready to load into a machine learning framework
    '''

    #TODO: In pipeline write this output to the catalogue
    # remove records the preceed the target period to have complete information:
    dataframe = dataframe.dropna() 
    #dataframe = dataframe.reset_index(drop = True) # we won't reset the index for now for traceability back to the date, ticker combination later after training

    # set the date as an index to us post-forecasting: This is a bad idea, come back to the concept
    #dataframe.set_index(keys = 'date', verify_integrity = False, inplace = True) # verify integrity Fale to allow duplicates**
    
    # remove fields that will not be used as predictive features (can be hardcoded since dataframe structure will be the same):
    dataframe = dataframe.drop(columns = [ 'date', 'high', 'low', 'open', 'volume', 'adj_close'])
    

    return dataframe


def identify_fields_to_standardize(dataframe: pd.DataFrame, parameters: Dict) -> np.array :

    '''creates a list of the continuous fields to standardize by dimension within the predictive model; NOTE: this is used within the standardizer
    
    Args:
        dataframe: dataframe that contains all of the fields of interest to be used in the calculations
        parameters:
            continuous_feature_cutoff: ratio of unique values to record count to be used to codify continuous features -> removes records from the standardization process which don't have enough data to standardize (e.g., boolean)

    Returns: list of continuous fields to use in the standardization process based on user's specifications of "uniqueness" threshold    

    '''

    numeric_fields = dataframe.select_dtypes(include = 'number').columns
    records = len(dataframe)

    record_summary = pd.DataFrame(dataframe[numeric_fields].nunique(), columns = ['unique_values'])
    record_summary['rows_in_df'] = records
    record_summary['value_to_record_ratio'] = record_summary['unique_values']/ record_summary['rows_in_df']

    # filter for a threshold specified by the user:
    record_summary = record_summary[record_summary['value_to_record_ratio'] > parameters['continuous_feature_cutoff']]

    # remove percentage features # TODO: later add in functionality to remove percentage based features

    return record_summary.index


# Justification for approach on scaling - the argument can be made that since our approach will generalize movemements across multiple securities that we need to standardize each security to its own price range.  Therefore, any features with price-relative values will be scaled per the security's price values to avoid odd splits in tree-based algos
# the concern with standardization is generally focused on not letting any one feature have considerably more weight in a model than another; however in this case, 


def standardize_continuous_features(dataframe: pd.DataFrame, parameters: Dict) -> pd.DataFrame:

    '''function that identifies the continuious features in the dataframe and standardizes each feature by equity to enable scaling relative to each equity
    
    Args:
        Dataframe: Pandas dataframe to be used in machine learning
        Parameters:
            stock_field: field indicating the stock for the window function to scan
            calculation_field: field for which the target is being calculated (used for drop in main row merge)
    
    Returns:
        Dataframe: containing the standardized data fields
    
    '''

    continuous_fields = list(identify_fields_to_standardize(dataframe = dataframe, parameters = parameters))

    # add in the ticker for grouping next:
    continuous_fields.append(parameters['stock_field'])

    # downselect to the fields that will be used to standardize:
    continuous_dataframe = dataframe[continuous_fields]

    # calculate z-scores: --> Standardizes within each feature to scale accordingly
    z_scores = (continuous_dataframe - continuous_dataframe.groupby(by = parameters['stock_field']).transform('mean')) / continuous_dataframe.groupby(by = parameters['stock_field']).transform('std')

    # drop the null ticker (not needed post groupby): 
    z_scores.drop(columns = [ parameters['stock_field'], parameters['calculation_field'] ], inplace = True)

    # rename the fields to indicate standardization:
    z_scores.columns = z_scores.columns + '_std'

    # drop original continuous fields # TODO: coming back after calculation checks:
    if parameters['drop_original_fields'] == True:
        continuous_fields.remove(parameters['stock_field'])
        dataframe.drop(columns = continuous_fields, inplace = True)

    # append the fields back into the core dataframe:
    z_scores = pd.concat([dataframe, z_scores], axis = 1)

    # remove the standardized target field:
    z_scores.drop(columns = z_scores.columns[z_scores.columns.str.contains('target')][1], inplace = True)

    # remove unnecessary items:
    del continuous_fields, continuous_dataframe

    return z_scores



def one_hot_encode_tickers(dataframe: pd.DataFrame, parameters: Dict) -> pd.DataFrame:

    '''Returns one-hot encoded features to the predictive dataset NOTE: May not work, but this retains some of the information in the original dataframe while also potentially giving the global model a nudge
       Note: we choose not to drop first for now, even though it's a trap; Can be used post processing or as model features
    Args:
        dataframe: core dataset that has been augmented with additional features
        parameters:
            stock_field: text field containing the 
    Returns:   
        dataframe with augmented columns
    
    '''

    dataframe = pd.get_dummies(data = dataframe, prefix = "ind", columns = [parameters['stock_field']], drop_first = False)

    return dataframe


def profile_target_variable(dataframe: pd.DataFrame, parameters: dict):


    '''Function that looks at the target variable and creates an output for the user to review and decide whether rebalancing will help classification task
    Args:
        dataframe: Main resulting dataframe from all data conversion steps
        parameters:
            
    
    '''
    # isolate the target variable:
    target_field = list(dataframe.columns[dataframe.columns.str.contains('target')])

    # create simple value count outputs:
    target_summary_table = pd.DataFrame(dataframe[target_field].value_counts()).reset_index()
    target_summary_table.rename(columns = {0 : 'counts'}, inplace = True)
    target_summary_table['proportion'] = target_summary_table['counts'] / target_summary_table['counts'].sum()

    # create bargraph and save it:
    ''' TODO : resolve ability to output a matplotlib plot in kedro catalog
    sns.countplot(x=target_field, data=dataframe)
    plt.title("Class Distribution")
    plt.show() '''
    target_field =', '.join(test.columns[test.columns.str.contains('target')].str.replace(r'\[|\]', ''))
    positive_proportion = target_summary_table[target_summary_table[target_field].astype(int) == 1]['proportion'].to_list()
   

    print('Classification target: ' + str(target_field) + " contains a class balance of: " + str(positive_proportion) + " in the positive case")
           

    return target_summary_table # TODO: Write this to the catalogue as a reporting output for the users



def create_training_test_splits(dataframe: pd.DataFrame, parameters: Dict) :

    '''Function that splits out training and test sets for machine learning; for the purposes of this model the way we piose the problem allows for random train test split
    Args:
        dataframe: pandas dataframe containing only the target field and the features to be used by the classifier
        parameters:
            test_ratio: proportion of samples in the dataframe to be used as a test set once the models are tuned and evaluated

    '''

    # define Y and x:
    target_feature = list(dataframe.columns[dataframe.columns.str.contains('target')])

    y = dataframe[target_feature]
    X = dataframe.drop(columns = target_feature)

    # create the training and test splits:
    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=parameters['test_size'], random_state=parameters['seed'], stratify = y)

    return X_train, X_test, y_train, y_test




In [69]:
df = catalog.load('combined_modeling_input')

[12/30/23 16:43:50] INFO     Loading data from 'combined_modeling_input' (CSVDataSet)...        ]8;id=183815;file:///opt/anaconda3/envs/stock-classification/lib/python3.7/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=368792;file:///opt/anaconda3/envs/stock-classification/lib/python3.7/site-packages/kedro/io/data_catalog.py#344\344]8;;\

In [70]:
# test: clean stock data:

df = clean_stock_data(dataframe = df)

In [71]:
df.head()


,close,ticker,7_close_sma,14_close_sma,21_close_sma,7_close_std,14_close_std,21_close_std,above_7_close_sma_ind,7_close_sma_pct_diff,above_14_close_sma_ind,14_close_sma_pct_diff,above_21_close_sma_ind,21_close_sma_pct_diff,cum_days_above_above_7_close_sma_ind,cum_days_above_above_14_close_sma_ind,cum_days_above_above_21_close_sma_ind,upper_bollinger_band,lower_bollinger_band,bol_pct_from_top,bol_pct_from_bottom,bol_range,bol_range_pct,target_20_days_ahead,target_20_days_ahead_ind
20,41.610001,AAPL,39.537500,38.988036,38.541548,1.378198,1.164633,1.335350,1.0,-0.049808,1.0,-0.063013,1.0,-0.073743,2.0,8.0,1.0,41.212247,35.870848,0.009651,0.159995,5.341399,0.0,43.742500,1
21,41.630001,AAPL,39.987500,39.242143,38.643929,1.485448,1.326377,1.484940,1.0,-0.039455,1.0,-0.057359,1.0,-0.071729,3.0,9.0,2.0,41.613808,35.674049,0.000389,0.166955,5.939758,0.0,43.962502,1
22,42.812500,AAPL,40.650000,39.621607,38.989881,1.573873,1.533400,1.571253,1.0,-0.050511,1.0,-0.074532,1.0,-0.089287,4.0,10.0,3.0,42.132386,35.847376,0.016142,0.194299,6.285010,0.0,43.882500,1
23,43.544998,AAPL,41.236428,39.998571,39.298452,1.796852,1.800364,1.794729,1.0,-0.053016,1.0,-0.081443,1.0,-0.097521,5.0,11.0,4.0,42.887911,35.708994,0.015321,0.219441,7.178918,0.0,43.630001,1
24,43.560001,AAPL,41.877143,40.343214,39.611667,1.694387,1.991546,1.938531,1.0,-0.038633,1.0,-0.073847,1.0,-0.090641,6.0,12.0,5.0,43.488729,35.734604,0.001639,0.218987,7.754125,0.0,43.125000,0


In [72]:
parameters = {'continuous_feature_cutoff' : 0.6,
              'stock_field' : 'ticker',
              'calculation_field' : 'close',
              'drop_original_fields' : True,
              'drop_stock_field': True, # keep this fixed 
              'test_size' : 0.20,
              'seed' : 1187,
              'cross_val_splits' : 5,
              'c' : 1.0,
              'kernel' : 'rbf',
              'gamma' : 'scale'
              
              }

In [73]:
# test: standardize features:
test = standardize_continuous_features(dataframe = df, parameters = parameters)



In [74]:
# one-hot encode: 
test = one_hot_encode_tickers(dataframe = test, parameters= parameters)

In [75]:
test['target_20_days_ahead_ind'][0:10]

20    1
21    1
22    1
23    1
24    0
25    1
26    1
27    1
28    1
29    1
Name: target_20_days_ahead_ind, dtype: int64

In [76]:
# train-test splits:
X_train, X_test, y_train, y_test = create_training_test_splits(dataframe=test, parameters= parameters)

In [77]:
y_train

,target_20_days_ahead_ind
1761,1
262,1
1029,1
176,1
1168,0
...,...
1547,0
1251,1
1644,0
1428,1


In [10]:
##################################### - Function development HERE

In [236]:
# function to profile the dataset target variable to determine whether we need to do some sort of rebalancing:

def profile_target_variable(dataframe: pd.DataFrame, parameters: dict):


    '''Function that looks at the target variable and creates an output for the user to review and decide whether rebalancing will help classification task
    Args:
        dataframe: Main resulting dataframe from all data conversion steps
        parameters:
            
    
    '''
    # isolate the target variable:
    target_field = list(dataframe.columns[dataframe.columns.str.contains('target')])

    # create simple value count outputs:
    target_summary_table = pd.DataFrame(dataframe[target_field].value_counts()).reset_index()
    target_summary_table.rename(columns = {0 : 'counts'}, inplace = True)
    target_summary_table['proportion'] = target_summary_table['counts'] / target_summary_table['counts'].sum()

    # create bargraph and save it:
    ''' TODO : resolve ability to output a matplotlib plot in kedro catalog
    sns.countplot(x=target_field, data=dataframe)
    plt.title("Class Distribution")
    plt.show() '''
    target_field =', '.join(test.columns[test.columns.str.contains('target')].str.replace(r'\[|\]', ''))
    positive_proportion = target_summary_table[target_summary_table[target_field].astype(int) == 1]['proportion'].to_list()
   

    print('Classification target: ' + str(target_field) + " contains a class balance of: " + str(positive_proportion) + " in the positive case")
           

    return target_summary_table # TODO: Write this to the catalouge as a reporting output for the users

    

[01/07/24 10:29:06] WARNING  /opt/anaconda3/envs/stock-classification/lib/python3.7/site-packages/i ]8;id=148812;file:///opt/anaconda3/envs/stock-classification/lib/python3.7/warnings.py\warnings.py]8;;\:]8;id=361863;file:///opt/anaconda3/envs/stock-classification/lib/python3.7/warnings.py#110\110]8;;\
                             pykernel_launcher.py:26: FutureWarning: The default value of regex                    
                             will change from True to False in a future version.                                   
                                                                                                                   

Classification target: target_20_days_ahead_ind contains a class balance of: [0.6210407239819005] in the positive case


[0.6210407239819005]

In [228]:
summary


,target_20_days_ahead_ind,counts,proportion
0,1,1647,0.621041
1,0,1005,0.378959


In [ ]:
############################ - Machine learning loop

In [78]:
# classifiers to use: support vector machine, decision tree, random forest, xgboost, adaboost

# TODO: Add in the confusion matrix values to make output analysis more straightforward

def train_models(X_train: pd.DataFrame, y_train: pd.DataFrame, parameters) -> pd.DataFrame:

    '''Trains a series of machine learning model outputs for evaluation by the user
    
    Args:
        X_train: inputs from train-test split function
        y_train: y-series from the train-test split function

    Returns:
        Summarized output of all ML models tried
    
    '''

    # define all of the models to be used:
    classifiers = {
    "LogisticRegression": LogisticRegression(n_jobs = -1, max_iter = 100000),
    "RandomForestClassifier": RandomForestClassifier(n_jobs = -1),
    "SVC": SVC(C = parameters['c'], kernel =parameters['kernel'], gamma = parameters['gamma']),
    "AdaBoostClassifier": AdaBoostClassifier()
    }

    # create a readable representation of the target:
    y_train = y_train.iloc[:, 0].values
    X_train = X_train.values

    #TODO: add precision, recall, f-measure on all sets
    #accuracies = {}
    names = []
    models = []
    fold = []
    training_samples = []

    #TODO: Rename test to validation **

    train_accuracies = []
    test_accuracies = []

    train_precisions = []
    test_precisions = []

    train_recalls = []
    test_recalls = []

    train_f_measures = []
    test_f_measures = []

    # iterate through the models:
    for name, classifier in classifiers.items():

        clf = classifier
        print(name)
        print (clf)

        # iterate through the folds: ->> not ideal to nest the loops here
        cv = StratifiedKFold(n_splits=parameters['cross_val_splits'], shuffle=True, random_state=parameters['seed']).split(X_train, y_train)

        for k, (fold_train, fold_test) in enumerate(cv):

             # append model name into list:
            models.append(str(classifier))
            
            clf.fit(X_train[fold_train],y_train[fold_train])
        

            # create predictions:
            train_pred = clf.predict(X = X_train[fold_train])
            test_pred = clf.predict(X = X_train[fold_test])

            # calculate accuracies:
            train_accuracy = clf.score(X_train[fold_train], y_train[fold_train])
            test_accuracy = clf.score(X_train[fold_test], y_train[fold_test])
    
            # calculate precision:
            train_precision = precision_score(y_train[fold_train], train_pred)
            test_precision = precision_score(y_train[fold_test], test_pred)
 
            # calculate recall:
            train_recall = recall_score(y_train[fold_train], train_pred)
            test_recall = recall_score(y_train[fold_test], test_pred)

            # calculate f-measure:
            train_f = f1_score(y_train[fold_train], train_pred)
            test_f = f1_score(y_train[fold_test], test_pred)
            

            # append name:
            names.append(name)
            
            # append training sample size:
            training_samples.append( len(X_train[fold_train]) )

            # append fold number to the list:
            fold.append(k+1)

            # append score into list:
            train_accuracies.append(train_accuracy)
            test_accuracies.append(test_accuracy)

            # append precisions to the list:
            train_precisions.append(train_precision)
            test_precisions.append(test_precision)

            # append recalls to the list:
            train_recalls.append(train_recall)
            test_recalls.append(test_recall)

            # append f-measures to the list:
            train_f_measures.append(train_f)
            test_f_measures.append(test_f)
    
    results_df = pd.DataFrame({
                "names" : names,
                "model" : models,
                "fold" : fold,
                "training_samples" : training_samples,
                "train_accuracy": train_accuracies,
                "test_accuracy": test_accuracies,
                "train_precision": train_precisions,
                "test_precision": test_precisions,
                "train_recall": train_recalls,
                "test_recall": test_recalls,
                "train_f_measures": train_f_measures,
                "test_f_measures": test_f_measures
                
                })

    # create aggregated results df:
    aggregated_results_df = results_df.drop(columns = ['fold']).groupby(by = ['names', 'model']).mean()
                            
    
    return results_df, aggregated_results_df #names, model, fold, train_accuracies, test_accuracies, train_precisions, test_precisions



In [64]:
##################################### - Testing functions HERE

In [79]:

#names, model, fold, train_accuracies, test_accuracies, train_precisions, test_precisions 

output, aggregated_output= train_models(X_train = X_train, y_train = y_train, parameters = parameters)
    

LogisticRegression
LogisticRegression(max_iter=100000, n_jobs=-1)
RandomForestClassifier
RandomForestClassifier(n_jobs=-1)
SVC
SVC()
AdaBoostClassifier
AdaBoostClassifier()


In [81]:
output

,names,model,fold,training_samples,train_accuracy,test_accuracy,train_precision,test_precision,train_recall,test_recall,train_f_measures,test_f_measures
0,LogisticRegression,"LogisticRegression(max_iter=100000, n_jobs=-1)",1,1696,0.635024,0.647059,0.655222,0.654891,0.869896,0.912879,0.747450,0.762658
1,LogisticRegression,"LogisticRegression(max_iter=100000, n_jobs=-1)",2,1697,0.654095,0.639151,0.668353,0.659942,0.878443,0.867424,0.759130,0.749591
2,LogisticRegression,"LogisticRegression(max_iter=100000, n_jobs=-1)",3,1697,0.643489,0.608491,0.661163,0.641399,0.873814,0.836502,0.752758,0.726073
3,LogisticRegression,"LogisticRegression(max_iter=100000, n_jobs=-1)",4,1697,0.636417,0.622642,0.651002,0.641096,0.893738,0.889734,0.753299,0.745223
4,LogisticRegression,"LogisticRegression(max_iter=100000, n_jobs=-1)",5,1697,0.641131,0.653302,0.655052,0.662921,0.891841,0.897338,0.755323,0.762520
5,RandomForestClassifier,RandomForestClassifier(n_jobs=-1),1,1696,1.000000,0.858824,1.000000,0.854167,1.000000,0.931818,1.000000,0.891304
6,RandomForestClassifier,RandomForestClassifier(n_jobs=-1),2,1697,1.000000,0.863208,1.000000,0.857639,1.000000,0.935606,1.000000,0.894928
7,RandomForestClassifier,RandomForestClassifier(n_jobs=-1),3,1697,1.000000,0.860849,1.000000,0.861702,1.000000,0.923954,1.000000,0.891743
8,RandomForestClassifier,RandomForestClassifier(n_jobs=-1),4,1697,1.000000,0.849057,1.000000,0.856631,1.000000,0.908745,1.000000,0.881919
9,RandomForestClassifier,RandomForestClassifier(n_jobs=-1),5,1697,1.000000,0.903302,1.000000,0.914179,1.000000,0.931559,1.000000,0.922787


In [68]:
aggregated_output

,,training_samples,train_accuracy,test_accuracy,train_precision,test_precision,train_recall,test_recall,train_f_measures,test_f_measures
names,model,,,,,,,,,
AdaBoostClassifier,AdaBoostClassifier(),1696.8,0.752711,0.702968,0.749999,0.713708,0.902813,0.871641,0.819266,0.784399
LogisticRegression,"LogisticRegression(max_iter=100000, n_jobs=-1)",1696.8,0.642031,0.634129,0.658158,0.652050,0.881546,0.880775,0.753592,0.749213
RandomForestClassifier,RandomForestClassifier(n_jobs=-1),1696.8,1.000000,0.865162,1.000000,0.866935,1.000000,0.925576,1.000000,0.895081
SVC,SVC(),1696.8,0.639322,0.631301,0.636062,0.631517,0.980446,0.975697,0.771486,0.766647


In [147]:
classifier = logistic_regression(X = X_train, y = y_train)

### Next to-dos:  
1.) Add parameters for all Classifiers to the parameters model  
2.) Add Select "n" best logic to the outputs
3.) Add in feature importances and feature selection before modeling run
3.) Add in Hypterparameter tuning
4.) Run with more positions/equity holdings